In [7]:
import pandas as pd
import numpy as np

""" 
# Play with the seed if need be
np.random.seed(42)
num_stocks = 80
industries = ['Tech', 'Healthcare', 'Finance', 'Energy', 'Consumer']

data = {
    'Stock Name': [f'Stock_{i+1}' for i in range(num_stocks)],
    'TSR': np.random.uniform(5, 15, num_stocks),  # TSR between 5% and 15%
    'Grade': np.random.uniform(50, 100, num_stocks),  # Grade between 50 and 100
    'Beta Value': np.random.uniform(0.5, 1.5, num_stocks),  # Beta between 0.5 and 1.5
    'Industry': np.random.choice(industries, num_stocks)  # Randomly assign industry
}

dummy_data = pd.DataFrame(data)
"""

" \n# Play with the seed if need be\nnp.random.seed(42)\nnum_stocks = 80\nindustries = ['Tech', 'Healthcare', 'Finance', 'Energy', 'Consumer']\n\ndata = {\n    'Stock Name': [f'Stock_{i+1}' for i in range(num_stocks)],\n    'TSR': np.random.uniform(5, 15, num_stocks),  # TSR between 5% and 15%\n    'Grade': np.random.uniform(50, 100, num_stocks),  # Grade between 50 and 100\n    'Beta Value': np.random.uniform(0.5, 1.5, num_stocks),  # Beta between 0.5 and 1.5\n    'Industry': np.random.choice(industries, num_stocks)  # Randomly assign industry\n}\n\ndummy_data = pd.DataFrame(data)\n"

In [8]:
# load your data
# something = pd.read_csv('something.csv')

companies_selected = pd.read_excel("Companies.xlsx")

In [9]:
# since TSR is in % we will grade wight in % as well
grade_weight = {"A": 90, "B": 80, "C": 70, "D": 60, "E": 50, "F": 40}
companies_selected["GradeWeight"] = companies_selected["Grade"].map(grade_weight)

In [10]:
def calculate_grade_score(data, x):
    return (1 - x) * data["TSR"] + x * data["GradeWeight"]

In [11]:
# default values set as: 16% for industry allocation and 4.5% mac allocation
# beta value for potfolio should be below 1
def monte_carlo_simulation(
    data, max_industry_allocation=0.16, target_beta=1.0, iterations=100
):

    best_portfolio = None
    best_portfolio_beta = float("inf")

    for _ in range(iterations):
        # The weightage we are giving x will range from 0.55 to 0.7
        # This will give us cases where our grade is not the main factor and TSR is which provides us with different scores on multiple iterations
        x = np.random.uniform(0.55, 0.7)

        # execute the function mention above
        data["Grade Score"] = data.apply(calculate_grade_score, axis=1, args=(x,))

        # obtain a weight scross all the investments. Basically getting a ratio so values for the weights will be between 0 and 1
        # basic normalisation
        data["Weight"] = data["Grade Score"] / data["Grade Score"].sum()

        # Lets focus on each industry
        # what is the current weight allocated to each industryxw
        data["Industry Allocation"] = data.groupby("Industry")["Weight"].transform(
            "sum"
        )
        data["Allocation_by_restriction"] = np.where(
            data["Industry Allocation"] > max_industry_allocation,
            max_industry_allocation,
            data["Industry Allocation"],
        )

        data["Normalised Weight"] = np.where(
            data["Allocation_by_restriction"] == 0.16,
            (data["Weight"] / data["Industry Allocation"]) * 0.16,
            data["Weight"],
        )

        # NEED TO ADD WEIGHT FOR THE MAX ALLOCATION

        # Calculate Portfolio Beta
        portfolio_beta = (data["Beta Value"] * data["Normalised Weight"]).sum()

        if portfolio_beta < target_beta:
            best_portfolio = data[
                [
                    "Company Name",
                    "Industry",
                    "Weight",
                    "Normalised Weight",
                    "Beta Value",
                ]
            ]
            best_portfolio_beta = portfolio_beta
            break

    return portfolio_beta, data

In [12]:
final_beta, ideal_portfolio = monte_carlo_simulation(companies_selected)

In [13]:
final_beta

0.5689451054135988

In [14]:
ideal_portfolio["Normalised Weight"].sum()

0.5292756105485721

In [15]:
ideal_portfolio.groupby("Industry")["Normalised Weight"].sum()

Industry
Consumer Staples    0.038519
Energy              0.062353
Finance             0.074946
Healthcare          0.073293
Industruals         0.034125
Real Estate         0.042096
Technology          0.160000
Utilities           0.043943
Name: Normalised Weight, dtype: float64

In [16]:
ideal_portfolio["final_weight"] = (
    ideal_portfolio["Normalised Weight"] / ideal_portfolio["Normalised Weight"].sum()
) * 100

In [17]:
round(ideal_portfolio["final_weight"].sum(), 2) == 100.00

True

In [18]:
ideal_portfolio

,Company Name,Industry,Beta Value,Grade,TSR,GradeWeight,Grade Score,Weight,Industry Allocation,Allocation_by_restriction,Normalised Weight,final_weight
0,NextEra Energy (NEE),Utilities,0.49,B,53.47,80,69.467292,0.004839,0.043943,0.043943,0.004839,0.914316
1,Duke Energy (DUK),Utilities,0.38,C,57.51,70,65.041330,0.004531,0.043943,0.043943,0.004531,0.856062
2,Atmos Energy (ATO),Utilities,0.50,B,54.13,80,69.729320,0.004858,0.043943,0.043943,0.004858,0.917765
3,Avangrid (AGR),Utilities,0.34,A,-8.57,90,50.866603,0.003543,0.043943,0.043943,0.003543,0.669497
4,Otter Tail (OTTR),Utilities,0.69,A,87.72,90,89.094814,0.006207,0.043943,0.043943,0.006207,1.172650
...,...,...,...,...,...,...,...,...,...,...,...,...
74,COSTCO WHOLESALE CORPORATION (XNAS:COST),Consumer Staples,0.79,D,17.43,60,43.099232,0.003002,0.038519,0.038519,0.003002,0.567264
75,DOLLAR GENERAL CORPORATION (XNYS:DG),Consumer Staples,0.45,B,14.09,80,53.832990,0.003750,0.038519,0.038519,0.003750,0.708540
76,COLGATE-PALMOLIVE COMPANY (XNYS:CL),Consumer Staples,0.41,A,15.40,90,60.382962,0.004206,0.038519,0.038519,0.004206,0.794750
77,THE HERSHEY COMPANY (XNYS:HSY),Consumer Staples,0.37,B,12.95,80,53.380397,0.003719,0.038519,0.038519,0.003719,0.702583
